# Шифр Цезаря: простой исторический метод шифрования

Шифр Цезаря – это один из самых простых исторических методов шифрования, который был использован еще в древности. Его название происходит от имени римского полководца Цезаря, который, по преданию, использовал этот метод для секретной переписки с своими генералами.

## Как работает шифр?

Шифр Цезаря основан на простом принципе сдвига символов в алфавите на определенное количество позиций. Каждая буква в сообщении заменяется другой буквой, находящейся на некотором фиксированном расстоянии в алфавите. Например, при сдвиге на 3 позиции, буква "А" станет "Г", "Б" станет "Д" и так далее.

Математически шифр Цезаря можно описать следующим образом:

- Пусть \( n \) – размер алфавита.
- Пусть \( x \) – исходный символ.
- Пусть \( k \) – сдвиг (ключ).
- Тогда зашифрованный символ \( y \) вычисляется по формуле: 
  \[ y = (x + k) \mod n \]

In [8]:
import math
from random import shuffle
import time
import torch

In [21]:
# Размер пакета для обучения модели
BATCH_SIZE = 10
# Длина строк в предложениях
STRING_SIZE = 60
# Количество эпох, через которые модель будет обучаться
NUM_EPOCHS = 20
# Скорость обучения модели
LEARNING_RATE = 0.05
# Путь к файлу с данными для обучения модели
FILE_NAME = "Datasets/ДЮНА.txt"
# Устройство, на котором будет происходить обучение
DEVICE = "cpu"
# Смещение для шифра Цезаря
CAESAR_OFFSET = 2

Так как шифр Цезаря работает на основе алфавита, необходимо создать аналогичный алфавит из доступных символов в тексте.
В качестве примера подготовки алфавита и набора данных для обучения модели мы возьмем текст книги Фрэнка Герберта - "Дюна", переведенной на русский язык.

In [33]:
class Alphabet(object):

    def __init__(self):
        # Пустая строка, куда будут добавляться символы из файла
        self.letters = ""

    def __len__(self):
        # Возвращает длину алфавита (количество символов)
        return len(self.letters)

    def __contains__(self, item):
        # Проверяет, содержится ли указанный символ в алфавите
        return item in self.letters

    def __getitem__(self, item):
        # Возвращает символ по индексу или находит индекс символа
        if isinstance(item, int):
            return self.letters[item % len(self.letters)]
        elif isinstance(item, str):
            return self.letters.find(item)

    def __str__(self):
        # Возвращает строковое представление алфавита
        letters = " ".join(self.letters)
        return f"Алфавит:\n {letters}\n {len(self)} символов"

    def load_from_file(self, file_path):
        # Загружает алфавит из файла
        with open(file_path, encoding="utf-8") as file:
            while True:
                text = file.read(STRING_SIZE)
                if not text:
                    break
                for ch in text:
                    # Добавляет символы в алфавит, если они не были добавлены ранее
                    if ch not in self.letters:
                        self.letters += ch
        return self


# Создание объекта алфавита и загрузка символов из файла
ALPHABET = Alphabet().load_from_file(FILE_NAME)
# Вывод алфавита на экран
print(ALPHABET)

Алфавит:
 С   с а м о г н ч л д п р е и т ь в ж з , б ы 
 у я к . Э й Б Г И М ш х Ш Ч О А П К ц Н щ - Д ю э З ъ ф : ? Е В Х Р Л " Т ! Я У Ж x ; Ф Ю ( ) 1 0 3 2 9 V I 8 5 Ц X 6 7 % Щ * Ь / Ы 4 Й Ъ L a s t m o d i f e S u n A g G M T
 111 символов


In [23]:
class SentenceDataset(torch.utils.data.Dataset):

    def __init__(self, raw_data, alphabet):
        # Конструктор класса
        super().__init__()
        # Длина датасета
        self._len = len(raw_data)
        # Кодирование символов входных данных с использованием алфавита
        self.y = torch.tensor(
            [[alphabet[ch] for ch in line] for line in raw_data]
        ).to(DEVICE)
        # Шифрование символов входных данных методом Цезаря
        self.x = torch.tensor(
            [[i + CAESAR_OFFSET for i in line] for line in self.y]
        ).to(DEVICE)
    
    def __len__(self):
        # Возвращает длину датасета
        return self._len

    def __getitem__(self, idx):
        # Возвращает элемент датасета по индексу idx
        return self.x[idx], self.y[idx]

In [26]:
def get_text_array(file_path, step):
    # Функция для чтения текста из файла и разделения его на массив строк определенной длины
    text_array = []
    with open(file_path, encoding="utf-8") as file:
        while True:
            text = file.read(STRING_SIZE)
            if not text:
                break
            text_array.append(text)
    # Удаление последнего элемента массива (если его длина меньше STRING_SIZE)
    del text_array[-1]
    return text_array

In [27]:
# Получение массива текста из файла и его перемешивание
raw_data = get_text_array(FILE_NAME, STRING_SIZE)
shuffle(raw_data)

# Выделение 10% данных для валидации
_10_percent = math.ceil(len(raw_data) * 0.1)
val_data = raw_data[:_10_percent]
raw_data = raw_data[_10_percent:]

# Выделение 20% данных для тестирования
_20_percent = math.ceil(len(raw_data) * 0.2)
test_data = raw_data[:_20_percent]
train_data = raw_data[_20_percent:]

# Кодирование данных для валидации
Y_val = torch.tensor([[ALPHABET[ch] for ch in line] for line in val_data])
X_val = torch.tensor([[i + CAESAR_OFFSET for i in line] for line in Y_val])

# Создание DataLoader для обучающего и тестового наборов данных
train_dl = torch.utils.data.DataLoader(
    SentenceDataset(
        train_data, ALPHABET
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True
)
test_dl = torch.utils.data.DataLoader(
    SentenceDataset(
        test_data, ALPHABET
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True
)

Наша рекуррентная нейронная сеть будет представлять собой довольно простую модель с слоем эмбеддинга, затем будет ячейка RNN и выходной линейный слой.
К размеру входных значений в слое эмбеддинга и размеру выходных значений линейного слоя необходимо добавить число, на которое мы будем сдвигать алфавит для шифрования текста.

In [28]:
class RNNModel(torch.nn.Module):
    
    def __init__(self):
        # Конструктор класса
        super().__init__()
        # Создание слоя Embedding для преобразования символов в вектора
        self.embed = torch.nn.Embedding(len(ALPHABET) + CAESAR_OFFSET, 32)
        # Создание слоя RNN с 128 скрытыми узлами
        self.rnn = torch.nn.RNN(32, 128, batch_first=True)
        # Создание линейного слоя для преобразования скрытых состояний RNN в выходные символы
        self.linear = torch.nn.Linear(128, len(ALPHABET) + CAESAR_OFFSET)

    def forward(self, sentence, state=None):
        # Прямой проход модели
        embed = self.embed(sentence)
        o, h = self.rnn(embed)
        return self.linear(o)

In [29]:
# Создание экземпляра модели RNN и перенос его на процессор
model = RNNModel().to(DEVICE)
# Определение функции потерь (CrossEntropyLoss) и перенос ее на устройство
loss = torch.nn.CrossEntropyLoss().to(DEVICE)
# Определение оптимизатора для обновления параметров модели с указанной скоростью обучения
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [30]:
# Цикл обучения модели на протяжении заданного количества эпох
for epoch in range(NUM_EPOCHS):
    # Инициализация переменных для хранения потерь, точности и количества итераций в текущей эпохе
    train_loss, train_acc, iter_num = .0, .0, .0
    start_epoch_time = time.time()
    
    # Переключение модели в режим обучения
    model.train()
    
    # Цикл по обучающему DataLoader
    for x_in, y_in in train_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        optimizer.zero_grad()
        # Прямой проход модели
        out = model.forward(x_in).view(-1, len(ALPHABET) + CAESAR_OFFSET)
        # Рассчет потерь
        l = loss(out, y_in)
        train_loss += l.item()
        # Рассчет точности
        batch_acc = (out.argmax(dim=1) == y_in)
        train_acc += batch_acc.sum().item() / batch_acc.shape[0]
        # Обратное распространение ошибки и обновление параметров модели
        l.backward()
        optimizer.step()
        iter_num += 1
    
    # Вывод информации о текущей эпохе после завершения обучения
    print(
        f"Epoch: {epoch}, loss: {train_loss:.4f}, accuracy: "
        f"{train_acc / iter_num:.4f}",
        end=" | "
    )
    
    # Инициализация переменных для хранения потерь, точности и количества итераций во время тестирования
    test_loss, test_acc, iter_num = .0, .0, .0
    
    # Переключение модели в режим оценки
    model.eval()
    
    # Цикл по тестовому DataLoader
    for x_in, y_in in test_dl:
        x_in = x_in
        y_in = y_in.view(1, -1).squeeze()
        # Прямой проход модели
        out = model.forward(x_in).view(-1, len(ALPHABET) + CAESAR_OFFSET)
        # Рассчет потерь
        l = loss(out, y_in)
        test_loss += l.item()
        # Рассчет точности
        batch_acc = (out.argmax(dim=1) == y_in)
        test_acc += batch_acc.sum().item() / batch_acc.shape[0]
        iter_num += 1
    
    # Вывод информации о тестировании после завершения эпохи
    print(
        f"test loss: {test_loss:.4f}, test accuracy: {test_acc / iter_num:.4f} | "
        f"{time.time() - start_epoch_time:.2f} sec."
    )

Epoch: 0, loss: 608.6864, accuracy: 0.9318 | test loss: 34.9112, test accuracy: 0.9821 | 17.67 sec.
Epoch: 1, loss: 94.4461, accuracy: 0.9909 | test loss: 14.5184, test accuracy: 0.9964 | 20.86 sec.
Epoch: 2, loss: 46.2364, accuracy: 0.9974 | test loss: 8.0755, test accuracy: 0.9986 | 20.90 sec.
Epoch: 3, loss: 28.8953, accuracy: 0.9984 | test loss: 5.4432, test accuracy: 0.9988 | 18.19 sec.
Epoch: 4, loss: 21.1267, accuracy: 0.9987 | test loss: 4.1436, test accuracy: 0.9990 | 20.64 sec.
Epoch: 5, loss: 16.9907, accuracy: 0.9988 | test loss: 3.3846, test accuracy: 0.9991 | 19.26 sec.
Epoch: 6, loss: 14.4262, accuracy: 0.9988 | test loss: 2.8801, test accuracy: 0.9991 | 18.62 sec.
Epoch: 7, loss: 12.6375, accuracy: 0.9988 | test loss: 2.5161, test accuracy: 0.9991 | 18.95 sec.
Epoch: 8, loss: 11.3044, accuracy: 0.9989 | test loss: 2.2346, test accuracy: 0.9992 | 19.18 sec.
Epoch: 9, loss: 10.2400, accuracy: 0.9990 | test loss: 2.0060, test accuracy: 0.9993 | 18.16 sec.
Epoch: 10, loss: 

In [31]:
# Выбор индекса для проверки результатов на валидационных данных
idx = 128

# Предсказание результатов на валидационных данных с помощью модели
val_results = model(X_val.to(DEVICE)).argmax(dim=2)

# Рассчет точности на валидационных данных
val_acc = (val_results == Y_val.to(DEVICE)).flatten()
val_acc = (val_acc.sum() / val_acc.shape[0]).item()

# Получение предсказанного предложения и истинного предложения из индекса idx
out_sentence = "".join([ALPHABET[i.item()] for i in val_results[idx]])
true_sentence = "".join([ALPHABET[i.item()] for i in Y_val[idx]])

# Вывод информации о точности на валидационных данных и сравнение предсказанного и истинного предложений
print(f"Точность на валидационных данных: {val_acc:.4f}")
print("-" * 20)
print(f"Предсказанное предложение:\n{out_sentence}")
print("-" * 20)
print(f"Истинное предложение:\n{true_sentence}")

Точность на валидационных данных: 0.9997
--------------------
Предсказанное предложение:
ое главное:
это -- канли, а ты не хуже моего знаешь правила.
--------------------
Истинное предложение:
ое главное:
это -- канли, а ты не хуже моего знаешь правила.


Поскольку шифр Цезаря является довольно примитивным способом шифрования текста, наша модель быстро обучается и показывает отличную точность на тестовых данных.

In [32]:
# Задание произвольного текста для шифрования
sentence = """Мудрость – это не только знание, но и способность применять его в жизни. 
Это навык видеть глубже и понимать суть вещей."""
# Преобразование текста в индексы алфавита
sentence_idx = [ALPHABET[i] for i in sentence]
# Шифрование текста с использованием шифра Цезаря
encrypted_sentence_idx = [i + CAESAR_OFFSET for i in sentence_idx]
encrypted_sentence = "".join([ALPHABET[i] for i in encrypted_sentence_idx])
# Преобразование зашифрованного текста обратно в токены алфавита
result = model(torch.tensor([encrypted_sentence_idx]).to(DEVICE)).argmax(dim=2)
deencrypted_sentence = "".join([ALPHABET[i.item()] for i in result.flatten()])
# Вывод зашифрованного и расшифрованного текста
print(f"Зашифрованное предложение: {encrypted_sentence}")
print("-" * 20)
print(f"Расшифрованное предложение: {deencrypted_sentence}")

Зашифрованное предложение: хкринмвжа аъвналтавнпжЭнаблольтыалнаьаменмн
лнмвжаеиьгтл.вжатчназа,ьбльйаяБвналозуЭазьртвжачпк
,таьаенльговжамквжазтДтГй
--------------------
Расшифрованное предложение: Мудрость Ч это не только знание, но и способность применять его в жизни. 
Это навык видеть глубже и понимать суть вещей.
